In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [8]:
#read the csv file
cwd = os.getcwd()
home_folder = os.path.dirname(cwd)
data_path = home_folder + '/srcTest/labelers_weight.txt'
print(data_path)
#set divider of cvs as ; and no coliumn names
df = pd.read_csv(data_path, sep=';', header=None)


#set the column names
df.columns = ['labeler', 'weight','lbl 0','lbl 1','lbl 2','lbl 3','lbl 255']
#set type of labeler as string
df['labeler'] = df['labeler'].astype(str)
#set type of weight as float
df['weight'] = df['weight'].astype(float)
df['weight'] = (df['weight']/(1024*1024))/(1024*1024)
#df['goodness'] = 1 - df['weight']
#set type of label as float

df['lbl 0'] = df['lbl 0'].astype(float)
df['lbl 1'] = df['lbl 1'].astype(float)
df['lbl 2'] = df['lbl 2'].astype(float)
df['lbl 3'] = df['lbl 3'].astype(float)
df['lbl 255'] = df['lbl 255'].astype(float)
#mean between the just the >0 labels
df['mean_positive'] =  df[['lbl 0', 'lbl 1', 'lbl 2', 'lbl 3', 'lbl 255']].apply(lambda row: row[row > 0].mean(), axis=1)
#put nan in the label thath are 0
df['lbl 0'] = df['lbl 0'].replace(0, np.nan)
df['lbl 1'] = df['lbl 1'].replace(0, np.nan)
df['lbl 2'] = df['lbl 2'].replace(0, np.nan)
df['lbl 3'] = df['lbl 3'].replace(0, np.nan)
df['lbl 255'] = df['lbl 255'].replace(0, np.nan)



d:\Universita\Master_Thesis\AutonomousDrivingOnMars/srcTest/labelers_weight.txt


In [9]:
import pandas as pd
from ydata_profiling import ProfileReport


profile = ProfileReport(df, title="Profiling Report")
profile.to_file("your_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

d:\AnacondaEnvs\gpu_2\Lib\site-packages\seaborn\matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)
d:\AnacondaEnvs\gpu_2\Lib\site-packages\ydata_profiling\model\missing.py:78: UserWarning: There was an attempt to generate the Heatmap missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(missing_diagrams={"Heatmap": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: '--'')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
#calculate the mean the medianm the mode and teh standard deviation
mean = df['weight'].mean()
median = df['weight'].median()
mode = df['weight'].mode()
std = df['weight'].std()

print('Mean: ', mean)
print('Median: ', median)
print('Standard Deviation: ', std)


Mean:  0.27064976166493615
Median:  0.22066555704389298
Standard Deviation:  0.1843937899797036


In [45]:
#import px to plot the data
import plotly.express as px

fig = px.histogram(df, x='weight', labels={'weight':'labeler score', 'count':'number of labelers'})

fig.add_vline(x=mean, line_width=3, line_dash="dash", line_color="green", annotation_text="Mean")
fig.add_vline(x=median, line_width=3, line_dash="dash", line_color="red", annotation_text="Median")
#show the standard deviation as a range make the annotation bold
fig.add_vrect(x0=max(0-0.005,mean-3*std), x1=mean+3*std, fillcolor="green", opacity=0.25, line_width=0, annotation_text="3 x Standard Deviation")

# distansiate the bars
fig.update_layout(bargap=0.1)
fig.show()


In [46]:

def detect_outliers(data):
    outliers=[]
    threshold=3
    mean = np.mean(data)
    std =np.std(data)
    
    for i in data:
        z_score= (i - mean)/std 
        if np.abs(z_score) > threshold:
            outliers.append(i)
    return outliers

In [7]:
import matplotlib.pyplot as plt
#plot the histogram of the weight
plt.hist(df['weight'], bins=10)
plt.title('Weight Histogram')
plt.xlabel('Weight')
plt.ylabel('Frequency')
plt.show()

C:\Users\usalv\AppData\Local\Temp\ipykernel_31328\2428040075.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [49]:
outliers = detect_outliers(df['weight'])
#istogram plot of the outliers
plt.hist(outliers, bins=10)
plt.show()
print(len(outliers)/len(df['weight'])*100)
print(len(outliers))
print(len(df['weight']))


1.2119626820123812
139
11469


C:\Users\usalv\AppData\Local\Temp\ipykernel_31328\3549980216.py:4: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



In [13]:
#drop the outliers
#df = df[~df['weight'].isin(outliers)]
df = df[df['weight'].isin(outliers)]

In [94]:
#save the cleaned data in a new csv file
df_save = df["labeler"]
#df_save.to_csv(home_folder + '/cleaned_labelers_weight.csv', sep=';', index=False)
df_save.to_csv(home_folder + '/outlier_labelers_weight.csv', sep=';', index=False, header=False)